```
실수 표현은 표현 범위가 넓은 대신 정밀도가 낮다.


        3.  실수
            1. 실수 연산의 함정
            2. 부동소수점
            3. 단정도와 배정도
            4. 1 바이트 실수 자료형 설계하기
                4.1 10진수 실수를 2진수 실수로 바꾸기
                4.2 정규화
                4.3 메모리 구조
                4.4 1바이트 부동소수점의 표현 범위
                4.5 1바이트 부동소수점의 정밀도
            5. 정밀도에 대한 고찰
                5.1 엡실론
                5.2 엡실론과 정밀도
            6. 마무리



1. 실수 연산의 함정
    
        # 1. 실수 연산의 함정

        a = 0.1
        result = 0.0    
        for i in range(100):
        result +=a

        print(result)

        ---------->
        9.99999999999998
    
        0.01 이 100번이니 1이 나오길 기대 하지만 결과는 예상과 달리 1에 가까운 수가 나온다. 

        a = 0.015625
        result = 0.0
        for i in range(100):
        result +=a
        print(result)       
        ---------->
        1.5625

        0.01 보다 복잡한 수여서 1.5625 보다 오차가 클 것 같지만 이번에는 정확한 1.5625 가 나온다. 
        
        이 결과를 이해하기 위해선 컴퓨터에서 실수를 표현하는 방법인 부동 소수점 이라는 개념을 이해해야 한다. 


2.  부동소수점 
    
    부동소수점은 소수점이 둥둥 떠다닌다는 뜻.
    
    가령 123.46을 다음과 같이 다양한 방법으로 표현 가능
    123.456 ---------->  1.23456 x 10^2
                                       12.3456 x 10^1
                                       1234.56 x 10^ (-1)


3. 단정도와 배정도 
        
        단정도 
            32비트( 8바이트 )
            부호 1비트, 지수부 8 비트, 가수부 23비트
            ()         ()()()()()()()()        ()()()()()()()()()()()()()()()()()()()()()()()

        배정도
            64비트(8바이트)
            부포 1비트, 지수부 11비트, 가수부 52비트 

        파이썬은 배정도를 사용. ( 단정도보다 훨씬 정밀도 높다.)
        
        sys 모듈 임포트 해서 확인해 보면 최댓값, 최솟값, 정밀도 등을 확인할 수 있다. 
        
        import sys
        sys.float_info
        ------------->
                sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.22507        38585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.22        0446049250313e-16, radix=2, rounds=1)
        
         # 배정도 부동수수점으로 표현할 수 있는 가장 큰 수 
            sys.float_info.max
            ---------->
                1.7976931348623157e+308

        # 배정도 부동소수점으로 표현할 수 있는 가장 작은 수 
        sys.float_info.min
        ----------->
       2.2250738585072014e-308


4. 1 바이트 실수 자료형 설계하기 
    
    실수 표현 방식을 쉽게 이해할 수 있도록   실수 자료형을 직접 만들어 보겠다. 
    다음은 실수 자료형을 표현한 수식
            
                +- 1.man x 2 ^ (exp - bias)

        1.man은 가수 (mantissa/fraction)
        2 는 밑수 
        exp - bias는 지수 (exponent)

        7.75 라는 10진수 실수를 1바이트 부동소수점으로 표현해 보겠다.
        
       4.1. 10진수 --> 2진수 
                7.75 -----> 111.11  ( 2^2 + 2^1 + 2^0 + 2^-1 + 2^-2)
        
        4.2. 정규화 
                왼쪽에 위치한 가수부분을 밑수보다 작은 자연수 되도록 만드는것.        
                2진수의 밑수는 2여서 보다 작은 자연수 1밖에 없다. 
                111.11 -----> 1.1111 x 2 ^2
             
        4.3. 메모리 구조
                정규화 된 1.1111 x 2^2를 앞 수식 +- 1.man x 2^(exp - bias) 와 비교해 보자.
                man --> 1111
                exp - bias --> 2
                이제 1 바이트의 메모리 구조를 정하고, man과 exp 값만 저장하면 설계가 끝난다. 
                이때 지수부, 가수부에 할당하는 비트 수에 따라 표현 범위와 정밀도가 결정된다. 
    
                        0    0000    000
                      부호  지수부    가수부 

                부호 --> 0, 1 
                지수부 --> exp값   (부호 비트 x 0~15의 양수 나타냄.)
                가수부 --> man값
        
                 bias는 지수의 부호 결정하는데 쓰임
                음수를 사용하려면 bias를 7로 두고 지수부(exp)에서 bias를 실제 지수로 사용한다. 
                bias  = 2 ^(n-1) -1  ( n 은 지수부 비트수. 여기서는 4 대입----> 7 )
                exp - bias (7)  = 2(실제지수) 이므로 
                exp = 9 
                
                부호  --> 0 
                지수부 --> 1001
                가수부 --> 1111
                
                문제는 가수부 3비트 할당인데 우리가 가진것은 4자리 ---> 맨 뒤 1 생략한다.
                
                0    1001     111     =    0100    1111 = 0x4f ( 실수 7.75를 1바이트 부동소수점으로 나타낸것.
        
        4.4 1바이트 부동 소수점의 표현 범위
                
                우리가 설계한 1바이트 부동소수점으로
                
                        표현할 수 있는 가장 작은 수 (지수부가 0001일 때) ---> 1.00 0 x 2^(-6)  = 0.015625
                        표현할 수 있는 가장 큰 수 (지수부가 1110 일때 ) ------> 1.111 x 2^7 = 240

                ( 단 지수부 비트가 모두 0 일때 (2^(-7)) 와 모두 1일 때 (2^8)는 0.0, 정규화 불가능, 무한대 NaN(Not a Number)같은 
                    특별한 상황을 나타내므로 제외한다. )

        4.5 1 바이트 부동 솟수점의 정밀도
                
                변환 과정에서 가수부 1111을 담을 공간이 부족해서 뒤의1을 표현 못했다. 
                0x4f      =       01001111 = 1.111x 2^2 =    111.1   =  7.5 는    7.75라는 실수를 완벽하게 표현 못한다. 
                0.25 만큼차이 나므로 그만큼 정밀도 떨어진다. 
        
이 블로그 참고 해라 .(바이어스 상수, 바이어스 표현법으로 -4의  16 부동소수법 값 구해봄.) 
                
        

5. 정밀도에 대한 고찰 
    
    5.1 엡실론
        실수 자료형에서 엡실론(epsilon)이란 1.0 과 그 다음으로 표현 가능한 수 (representable float) 사이의 차이를 말한다.

    
        import sys
        sys.float_info.epsilon
        ---------->
        2.220446049250313e-16

        sys 모듈의 float_info에 있는 epsilon을 출력한 결과.  
        이 값이 어떻게 나왔을까? 
        
        3절에서 파이썬이 사용하는 배정도(double)의 가수부가 52비트라 했다. 
        1.0을 배정도에 맞춰 표현하면 다음과 같다. 
        
                1.0000 ......0000( 0:52개) x 2^0

        배정도에서 1.0 다음으로 표현할 수 있는 수(representable float)는 다음과 같다.
            
                1.0000......00000(0:51개 , 1:마지막 비트 ) x 2^0

        따라서 두수의 차이는 
                
                0.00000.....00000(0:51개 , 마지막 비트1) x 2^0

        이를 10진수로 바꾸면 엡실론이 나온다. 

                2.220446049250313e-16
    5.2 엡실론과 정밀도 
        
            엡실론을 이용하면 그 실수 다음에 표현할 수 있는 수를 알아낼 수 있다. 
            배정도 실수 9.25를 부동소수점 방식으로 표현하면 1.00101 x 2^3
            여기서 지수부분만 떼어 엡실론을 곱하면
            이 실수와 다음 표현 가능한 수 사이의 차이 구할수 있다.


# 5.2 엡실론과 정밀도
 
import sys
ep = sys.float_info.epsilon
a = 9.25
diff = (2**3)*ep
diff







Out[10]:
1.7763568394002505e-15
b = a+diff
b


Out[12]:
9.250000000000002

diff는 지수부분인 2^3 에 엡실론을 곱한 값으로 9.25와 그 다음 표현 가능한 수 사이의 차이이다. 
b 는 a에 diff를 더했으므로 9.25 다음에 표현 가능한 수를 나타낸다. 


# 5.2 엡실론과 정밀도 
 
import sys
ep = sys.float_info.epsilon
diff = (2**3)*ep
diff
a = 9.25
half_diff = diff/2
 
c = a + half_diff
a == c
 


Out[13]:
True

diff보다 작은 값을 더한 수를 부동소수점 방식에서는 표현 할 수 없다.  = 정밀도가 떨어진다. 

 
a = (2.0) ** 53
a
 


Out[15]:
9007199254740992.0
 
diff_new = (2**53)*ep
diff_new


Out[17]:
2.0
 
b = a+1.0
a == b
 


Out[18]:
True


여기서도 마찬가지. diff 가 2.0 이라서 
그보다 작은 1.0을 a 에 더해주면 그 차이를 표현 못한다. 




마무리.

부동 소수점은 지수부에 따라 아주 작은 수와 아주 큰 수를 표현할 수 있지만 어떤 상황에서는 16, 17과 큰 단위의 정수조차 제대로 표현할 수 없을 때도 있다. 
표현범위는 넓지만 정밀도는 낮다. 

```

In [ ]:
# 1. 실수 연산의 함정

a = 0.1
result = 0.0
for i in range(100):
    result +=a

print(result)

In [4]:
a = 0.015625
result = 0.0
for i in range(100):
    result +=a

print(result)

1.5625


In [5]:
# 3. 단정도와 배정도

import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

In [6]:
# 배정도 부동수수점으로 표현할 수 있는 가장 큰 수 

sys.float_info.max

1.7976931348623157e+308

In [7]:
# 배정도 부동소수점으로 표현할 수 있는 가장 작은 수 

sys.float_info.min

2.2250738585072014e-308

In [9]:
# 5. 정밀도에 대한 고찰
    
# 5.1 엡실론
    
import sys
sys.float_info.epsilon

2.220446049250313e-16

In [10]:
# 5.2 엡실론과 정밀도

import sys
ep = sys.float_info.epsilon
a = 9.25
diff = (2**3)*ep
diff

1.7763568394002505e-15

In [12]:
b = a+diff
b

9.250000000000002

In [13]:
# 5.2 엡실론과 정밀도 

import sys
ep = sys.float_info.epsilon
diff = (2**3)*ep
diff
a = 9.25
half_diff = diff/2

c = a + half_diff
a == c


True

In [15]:
a = (2.0) ** 53
a


9007199254740992.0

In [17]:
diff_new = (2**53)*ep
diff_new

2.0

In [18]:
b = a+1.0
a == b


True